# Identification (Accès restreint)

**IMPORTANT :** Demandez une autorisation sur : https://aka.ms/azure-speaker-recognition. Renseignez `SERVEUR` avec le serveur de votre point de terminaison (endpoint) et `PROFIL` avec l'identifiant de profil obtenu à l'étape précédente. Récupérez également la clé et renseignez là lors de l'exécution de la première cellule.

In [1]:
from getpass import getpass

CLE = getpass("Clé")
SERVEUR = 'https://francecentral.api.cognitive.microsoft.com'  # Votre serveur de endpoint

PROFIL = ''

## Tentative d'identification

In [ ]:
import requests

reponse = requests.post(
    url=f'{SERVEUR}/speaker-recognition/identification/text-independent/profiles:identifySingleSpeaker?api-version=2021-09-05&profileIds={PROFIL}',
    headers={'Ocp-Apim-Subscription-Key': CLE},
    json={'locale': 'fr-FR'}
)
resultat = reponse.json()
resultat

In [ ]:
if reponse.status_code == 200:
  if resultat["identifiedProfile"]["score"]<.40:
    adverbe = ' peut-être'
  elif resultat["identifiedProfile"]["score"]<.70:
    adverbe = ' sûrement'
  else:
    adverbe = ''
  print(f"Oui, c'est{adverbe} lui !")
else:
  print("Non ce n'est pas lui.")
